In [10]:
import json
import pandas as pd

In [11]:
folder = 'F:\\_Bounty\\northwell\\input_files\\'
file = "11-1639818_Mather Memorial Hospital_StandardCharges.json"

In [28]:
with open(folder + file, 'r') as f:
    jf = json.load(f)

In [29]:
try:
        cols = jf['Headers']
        data = jf['Data']
        df = pd.DataFrame(columns=cols, data=data)
        normal_file = True
except TypeError:
    df = pd.DataFrame(jf)
    normal_file = False
del jf

In [32]:
print(df.columns.tolist())

['Site', 'Identifier_Code', 'Billing_Code', 'Identification_Description', 'Charge', 'GHI HMO OP', 'GHI HMO IP', 'Health Care Partners HMO OP', 'Health Care Partners HMO IP', 'HIP EPO OP', 'HIP EPO IP', 'HIP HMO OP', 'HIP HMO IP', 'HIP HMO ACP OP', 'HIP HMO ACP IP', 'HIP Child Health Plus OP', 'HIP Child Health Plus IP ', 'GHI Federal Employees OP', 'GHI Federal Employees IP', 'GHI Network Access OP', 'GHI Network Access IP', 'GHI NYC Health OP', 'GHI NYC Health IP', 'GHI PPO OP', 'GHI PPO IP', 'HIP PPO OP', 'HIP PPO IP', 'Humana OP', 'Humana IP', 'HCP Select Care HIX OP', 'HCP Select Care HIX IP', 'HIP Select Care OP', 'HIP Select Care IP', 'Health Care Partners Essential 1 or 2 OP', 'Health Care Partners Essential 1 or 2 IP', 'HIP Essential 1 or 2 OP', 'HIP Essential 1 or 2 IP', 'Aetna HMO OP', 'Aetna HMO IP', 'Aetna Managed Choice POS OP', 'Aetna Managed Choice POS IP', 'Aetna Network OP', 'Aetna Network IP', 'Aetna PPO OP', 'Aetna PPO IP', 'Aetna Signature OP', 'Aetna Signature IP',

In [15]:
df.rename(columns={
    'Identification_Description': 'description',
    'Identifier_Description': 'description',
    'Identifier_Code': 'local_code',
    'Billing_Code': 'code'
}, inplace=True)

In [16]:
df.drop('Site', axis=1, inplace=True)

In [17]:
df['local_code'] = df['local_code'].replace("", pd.NA)
df['code'] = df['code'].replace("", pd.NA)

In [18]:
cols = df.columns.tolist()
id_vars = cols[:3]
value_vars = cols[3:]

df = pd.melt(df, id_vars=id_vars, value_vars=value_vars, var_name='payer_name', value_name='standard_charge')

In [19]:
df['standard_charge'] = df['standard_charge'].replace('', pd.NA)
df['standard_charge'] = df['standard_charge'].str.replace(',', '')

In [20]:
df.dropna(subset='standard_charge', inplace=True)

In [21]:
mapping = {
    'Charge': 'gross',
    'De-identified Maximum': 'max',
    'De-identified Minimum': 'min',
    'Discounted Cash Price': 'cash',   
}

df['payer_category'] = df['payer_name'].map(mapping).fillna('payer')

In [22]:
df.loc[~df['code'].isna()]

,local_code,code,description,payer_name,standard_charge,payer_category
0,-,APR DRG 0011,Liver transplant &/or intestinal transplant,Charge,-,gross
1,-,APR DRG 0012,Liver transplant &/or intestinal transplant,Charge,-,gross
2,-,APR DRG 0013,Liver transplant &/or intestinal transplant,Charge,-,gross
3,-,APR DRG 0014,Liver transplant &/or intestinal transplant,Charge,-,gross
4,-,APR DRG 0021,Heart &/or lung transplant,Charge,-,gross
...,...,...,...,...,...,...
5833627,81730350,90840,Psytx crisis ea addl 30 min,De-identified Maximum all,319.00,payer
5833628,81730087,90846,Family psytx w/o pt 50 min,De-identified Maximum all,319.00,payer
5833629,81730095,90847,Family psytx w/pt 50 min,De-identified Maximum all,319.00,payer
5833630,81730103,90849,Multiple family group psytx,De-identified Maximum all,319.00,payer


In [26]:
df.loc[df['standard_charge'].str.lower() == 'case rate']

,local_code,code,description,payer_name,standard_charge,payer_category
19031,68010099,96401,CHEMO INJ NON-HOR ANTI-NEO,GHI HMO OP,Case Rate,payer
19032,68010628,96402,CHEMO INJ HORMONAL ANTI-NEO,GHI HMO OP,Case Rate,payer
19033,68010289,96409,IV PUSH TECH 1ST CHMO DRG 1HR,GHI HMO OP,Case Rate,payer
19034,68010636,96411,IV PUSH TECH EA AD'L CHEMO DRG,GHI HMO OP,Case Rate,payer
19035,68010297,96413,IV INFUSION TECH 1ST CHMO DRUG,GHI HMO OP,Case Rate,payer
...,...,...,...,...,...,...
991311,68010321,96422,IA INFUS CHEMO 1ST HR 96422,GEHA OP,Case Rate,payer
991312,68010388,96423,IA INFUS CHEMO EA ADD HR 96423,GEHA OP,Case Rate,payer
991313,68010339,96425,"CHEMO,IV>8 HOURS PORTABLE PUMP",GEHA OP,Case Rate,payer
991315,68010347,96542,CHEMO INJECTN INTO A RESERVOIR,GEHA OP,Case Rate,payer


In [ ]:
df.loc[(~df['code'].isna()) & (df['code'].str.contains('APR DRG')), 'apr_drg_temp'] = df['code']
df.loc[~df['apr_drg_temp'].isna(), 'apr_drg_temp'] = df['apr_drg_temp'].str.replace('APR DRG ', '')
df.loc[~df['apr_drg_temp'].isna(), 'apr_drg'] = df['apr_drg_temp'].str[:3] + '-' + df['apr_drg_temp'].str[-1]
df.drop('apr_drg_temp', axis=1, inplace=True)


In [ ]:
df.loc[(~df['code'].isna()) & (df['code'].str.match(r'^[A-Z][0-9]{4}|[0-9]{5}|[0-9]{4}[A-Z]$')), 'hcpcs_cpt'] = df['code']

In [ ]:
ccn = {'11-1667761_South Shore University Hospital_StandardCharges.json': '330043',
 '11-1630914_Huntington Hospital_StandardCharges.json': '330045',
 '11-1562701_North Shore University Hospital_StandardCharges.json': '330106',
 '11-1661359_Peconic Bay Medical Center_StandardCharges.json': '330107',
 '13-1624070_Lenox Hill Hospital_StandardCharges.json': '330119',
 '11-2868878_Staten Island University Hospital_StandardCharges.json': '330160',
 '13-1740118_Northern Westchester Hospital_StandardCharges.json': '330162',
 '11-1633487_Glen Cove Hospital_StandardCharges.json': '330181',
 '11-1639818_Mather Memorial Hospital_StandardCharges.json': '330185',
 '11-2241326_Long Island Jewish Hospital_StandardCharges.json': '330195',
 '13-1725076_Phelps Memorial Hospital Center_StandardCharges.json': '330261',
 '11-3241243_Plainview Hospital_StandardCharges.json': '330331',
 '11-2241326_Long Island Jewish Forest Hills_StandardCharges.json': '330353',
 '11-2241326_Long Island Jewish Valley Stream_StandardCharges.json': '330372',
 '11-1562701_Syosset Hospital_StandardCharges.json': '330398'}

In [ ]:
id = ccn[file]

df['hospital_id'] = id

# df.to_csv(file.split('_')[0] + '_' + id + '.csv', index=False)

In [ ]:
mask = pd.to_numeric(df['standard_charge'], errors='coerce').isna()
df2 = df[mask]
df2.drop_duplicates(subset='standard_charge')

,local_code,code,description,payer_name,standard_charge,payer_category,apr_drg,hcpcs_cpt,hospital_id,ms_drg,contracting_method,additional_payer_notes
55528,4100981,81232,Dpyd gene common variants,Empire Blue Access EPO,<NA>,payer,NaN,81232,330045,NaN,other,POC


In [ ]:
df = df.drop(df[df['standard_charge'] == 'N/A'].index)

In [ ]:
df.loc[(~df['code'].isna()) & (df['code'].str.contains('MSDRG')), 'ms_drg'] = df['code'].str.extract(r'MSDRG(\d+)', expand=False)

In [ ]:
# Create a dictionary mapping of standard_charge values to replacement values
charge_map = {
    'POC': ('other', 'POC', pd.NA),
    'NSP': ('other', 'NSP', pd.NA),
    'Packaged': ('other', 'Packaged', pd.NA),
    'APG - Bundling': ('other', 'APG - Bundling', pd.NA)
}

# Replace standard_charge values based on the dictionary mapping
for charge, values in charge_map.items():
    df.loc[df['standard_charge'] == charge, ['contracting_method', 'additional_payer_notes', 'standard_charge']] = values

# Replace standard_charge values that contain ' PD'
df.loc[df['standard_charge'].str.contains(' PD', na=False), ['contracting_method', 'additional_payer_notes']] = ['other', 'PD']
df['standard_charge'] = df['standard_charge'].str.replace(' PD', '')

In [ ]:
# df.loc[df['standard_charge'] == 'POC', ['contracting_method', 'additional_payer_notes', 'standard_charge']] = ['other', 'POC', pd.NA]
# df.loc[df['standard_charge'] == 'NSP', ['contracting_method', 'additional_payer_notes', 'standard_charge']] = ['other', 'NSP', pd.NA]
# df.loc[df['standard_charge'] == 'Packaged', ['contracting_method', 'additional_payer_notes', 'standard_charge']] = ['other', 'Packaged', pd.NA]
# df.loc[df['standard_charge'] == 'APG - Bundling', ['contracting_method', 'additional_payer_notes', 'standard_charge']] = ['other', 'APG - Bundling', pd.NA]

In [ ]:
# df.loc[(~df['standard_charge'].isna()) & (df['standard_charge'].str.contains('PD')), ['contracting_method', 'additional_payer_notes']] = ['other', 'PD']
# df['standard_charge'] = df['standard_charge'].str.replace(' PD', '')

In [ ]:
df.loc[df['code'].str.len() == 5]

,local_code,code,description,payer_name,standard_charge,payer_category,apr_drg,hcpcs_cpt,hospital_id,ms_drg,contracting_method,additional_payer_notes
1819,4210239,78300,Bone imaging limited area,Charge,1316.00,gross,NaN,78300,330045,NaN,NaN,NaN
1820,4210248,78709,K flow/funct image multiple,Charge,2950.00,gross,NaN,78709,330045,NaN,NaN,NaN
1821,4210410,78597,Lung perfusion differential,Charge,1632.00,gross,NaN,78597,330045,NaN,NaN,NaN
1822,4210420,78598,Lung perf&ventilat diferentl,Charge,2950.00,gross,NaN,78598,330045,NaN,NaN,NaN
1823,4210700,78102,Bone marrow imaging ltd,Charge,1086.00,gross,NaN,78102,330045,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
14483237,64410590,J1566,immune globulin (GAMMAGARD S/D 10%) 5 Gram(s) ...,Discounted Cash Price,1212.97,cash,NaN,J1566,330045,NaN,NaN,NaN
14483242,64411564,J9218,leuprolide Kit (LUPRON) 5 mG/mL Injectable,Discounted Cash Price,916.21,cash,NaN,J9218,330045,NaN,NaN,NaN
14483246,64411937,J3490,"lidocaine 1% w/epinephrine 1:100,000 Injection",Discounted Cash Price,0.07,cash,NaN,J3490,330045,NaN,NaN,NaN
14483247,64411945,J3490,"lidocaine 1.5% w/epinephrine 1:200,000 Injectable",Discounted Cash Price,0.31,cash,NaN,J3490,330045,NaN,NaN,NaN
